In [1]:
import numpy as np
import os
import csv
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
import dash
from dash import Dash, dcc, html, Input, Output, callback, dash_table, State, MATCH, ALL, Patch, State, ctx, register_page, page_container, no_update
import dash_bootstrap_components as dbc
import time
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import pymysql
import dash_daq as daq
import datetime
import time

In [2]:
# Стили для первого блока
FIRST_BLOCK = {
    'width': '40%',  # Ширина блока 40%
    'height': '50%',  # Примерная высота блока
    'background-color': 'white',  # Цвет фона блока
    'border-radius': '20px',  # Скругленные углы
    'box-shadow': '0 4px 15px rgba(0, 0, 0, 0.3)',  # Тень блока
    'padding': '10px 20px',  # Внутренние отступы
}

# Стили для второго блока
SECOND_BLOCK = {
    'width': '40%',  # Ширина блока 40%
    'height': '50%',  # Примерная высота блока
    'background-color': 'white',  # Цвет фона блока
    'border-radius': '20px',  # Скругленные углы
    'box-shadow': '0 4px 15px rgba(0, 0, 0, 0.3)',  # Тень блока
}

# Стили для третьего блока
THIRD_BLOCK = {
    'width': '90%',  # Ширина блока 90% (с учетом отступов по бокам)
    'height': '50%',  # Примерная высота блока
    'background-color': 'white',  # Цвет фона блока
    'border-radius': '20px',  # Скругленные углы
    'box-shadow': '0 4px 15px rgba(0, 0, 0, 0.3)',  # Тень блока
    'margin-left': '5%',  # Отступ слева
    'margin-top': '5%',  # Отступ сверху
    'margin-bottom': '5%',  # Отступ снизу
}

# Контейнер для первого и второго блоков в ряд
ROW_CONTAINER = {
    'display': 'flex',  # Используем Flexbox для выравнивания блоков по горизонтали
    'justify-content': 'space-between',  # Размещение блоков с равными промежутками
    'width': '90%',  # Ширина контейнера
    'margin-left': '5%',  # Отступ слева для контейнера
    'margin-top': '5%',  # Отступ сверху для контейнера
}

# Стили для первых трёх блоков, которые будут располагаться в ряд
ROW_BLOCK = {
    'display': 'flex',
    'margin-top': '2%',  # Отступ сверху
    'margin-left': '10%',  # Отступ слева
    'margin-right': '2%',  # Отступ слева
    'width': '80%',  # Каждый блок будет занимать часть ширины родительского блока
    'justify-content': 'space-between',  # Размещение с равными промежутками
    'margin-bottom': '10px'  # Отступ снизу, чтобы отделить от четвёртого блока
}

# Стили для отдельного блока внутри ряда
BLOCK_STYLE = {
    'width': '30%',  # Каждый блок занимает 30% ширины ряда
    'height': '50%',  # Высота каждого блока
    'background-color': 'white',  # Цвет фона
    'border-radius': '10px',  # Скругленные углы
}

# Стили для четвёртого блока, который идёт под первым рядом
FOURTH_BLOCK_STYLE = {
    'width': '80%',  # Занимает всю ширину контейнера
    'margin-left': '10%',  # Отступ слева
    'margin-right': '2%',  # Отступ слева
    'height': '145px',  # Высота блока
    'background-color': '#A9A9A9',  # Цвет фона
    'border-radius': '10px',  # Скругленные углы
}

In [3]:
app = dash. Dash(__name__,  title='Робоадвайзинг - профиль', suppress_callback_exceptions=True)

In [4]:
app.layout = html.Div([ 
                        html.H1(children = 'Профиль пользователя',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),  
                        html.Div([
                                     html.Div([
                                                  html.H2(children = 'Риск профиль клиента',
                                                            style = {
                                                                     'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                                    }
                                                            ),
                                                  html.H4(children = 'Тип портфеля: ',
                                                            style = {
                                                                     'textAlign': 'left',  'color': '#B2B3B5', 'font-weight': '2'
                                                                    }
                                                            ),  
                                                  html.H4(children = 'Услуги миграционного консалтинга: ',
                                                            style = {
                                                                     'textAlign': 'left',  'color': '#B2B3B5', 'font-weight': '2'
                                                                    }
                                                            ), 
                                                  html.H4(children = 'Услуги страхования: ',
                                                            style = {
                                                                     'textAlign': 'left',  'color': '#B2B3B5', 'font-weight': '2'
                                                                    }
                                                            ),   
                                                  html.H4(children = 'Сумма инвестирования в год: ',
                                                            style = {
                                                                     'textAlign': 'left',  'color': '#B2B3B5', 'font-weight': '2'
                                                                    }
                                                            ),  
                                                 ],style=FIRST_BLOCK),
                                     html.Div([
                                                  html.Div([
                                                                html.Div([
                                                                            html.H4(children = 'Клиент',
                                                                                    style = {
                                                                                             'textAlign': 'left',  'color': '#333332', 'font-weight': '2'
                                                                                            }
                                                                                    ),  
                                                                            ],style=BLOCK_STYLE),  # Первый блок
                                                                html.Div([
                                                                            html.H4(children = 'Срок накопления',
                                                                                    style = {
                                                                                             'textAlign': 'left',  'color': '#333332', 'font-weight': '2'
                                                                                            }
                                                                                    ), 
                                                                            ],style=BLOCK_STYLE),  # Второй блок
                                                                html.Div([
                                                                            html.H4(children = 'Сумма инвестиций',
                                                                                    style = {
                                                                                             'textAlign': 'left',  'color': '#333332', 'font-weight': '2'
                                                                                            }
                                                                                    ), 
                                                                            ],style=BLOCK_STYLE),  # Третий блок
                                                            ], style=ROW_BLOCK),
            
                                                # Четвёртый блок под ними
                                                html.Div(style=FOURTH_BLOCK_STYLE)
                                                                             ],style=SECOND_BLOCK),
                                ], style=ROW_CONTAINER),
                         html.Div([
                                      html.H2(children = 'Цели клиента',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),
                                     ],style=THIRD_BLOCK),
    
                      ])
if __name__ == '__main__':
    app.run_server(port=1114, host="0.0.0.0")